<center>
<h4>CDS 110, Lecture 4a</h4>
<font color=blue><h1>Dynamics and State Feedback Control of a Predator-Prey Model</h1></font>
<h3>Richard M. Murray and Natalie Bernat, Winter 2024</h3>
</center>
<br>

In this lecture we describe the use of state space control concepts to analyze and stabilize the dynamics of a nonlinear model of a predator-prey system.


In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

try:
  import control as ct
  print("python-control version:", ct.__version__)
except ImportError:
  # Get the development version, which fixes a bug that affects the code below
  !pip install git+https://github.com/python-control/python-control.git
  import control as ct

## Predator-Prey System Model

[add pictures + description]

In [ ]:
# Define the dynamics for the predator-prey system (no input)
predprey_params = {'r': 1.6, 'd': 0.56, 'b': 0.6, 'k': 125, 'a': 3.2, 'c': 50}
def predprey_update(t, x, u, params):
    """Predator prey dynamics"""
    r, d, b, k, a, c = map(params.get, ['r', 'd', 'b', 'k', 'a', 'c'])
    u = np.atleast_1d(u)    # Fix python-control bug
    u = np.clip(u, -r, r)

    # Dynamics for the system
    dx0 = (r + u[0]) * x[0] * (1 - x[0]/k) - a * x[1] * x[0]/(c + x[0])
    dx1 = b * a * x[1] * x[0] / (c + x[0]) - d * x[1]

    return np.array([dx0, dx1])

# Create a nonlinear I/O system
predprey = ct.nlsys(
    predprey_update, name='predprey', params=predprey_params,
    states=['H', 'L'], inputs='u', outputs=['H', 'L'])

### Open loop dynamics

[Add description]

In [ ]:
T = np.linspace(0, 100, 500)
response = ct.input_output_response(
    predprey, T, 0, [35, 35]
)
ct.time_response_plot(response, plot_inputs=False, overlay_signals=True)

In [ ]:
# Generate a simple phase portrait
ct.phase_plane_plot(predprey, [0, 120, 0, 100], 0.1, gridtype='meshgrid');

In [ ]:
# Generate a phase portrait
ct.phaseplot.equilpoints(predprey, [-5, 126, -5, 100])
ct.phaseplot.streamlines(
    predprey, np.array([
        [0, 100], [1, 0],
    ]), 10, color='b')
ct.phaseplot.streamlines(
    predprey, np.array([[124, 1]]), np.linspace(0, 10, 500), color='b')
ct.phaseplot.streamlines(
    predprey, np.array([[125, 25], [125, 50], [125, 75]]), 3, color='b')
ct.phaseplot.streamlines(predprey, np.array([2, 8]), 6, color='b')
ct.phaseplot.streamlines(
    predprey, np.array([[20, 30]]), np.linspace(0, 65, 500),
    gridtype='circlegrid', gridspec=[2, 1], arrows=10, color='r')
ct.phaseplot.vectorfield(predprey, [5, 125, 5, 100], gridspec=[20, 20])

# Add the limit cycle
resp1 = ct.initial_response(predprey, np.linspace(0, 100), [20, 75])
resp2 = ct.initial_response(
    predprey, np.linspace(0, 20, 500), resp1.states[:, -1])
plt.plot(resp2.states[0], resp2.states[1], color='k')


### Find the equilibrium points and check stability

In [ ]:
xe, ue = ct.find_eqpt(predprey, [20, 30], 0)
print(f"{xe=}")
print(f"{ue=}")

In [ ]:
sys = predprey.linearize(xe, ue)
print(sys)
print("Poles: ", sys.poles())

## Stabilization

[Fill in]

In [ ]:
K = ct.place(sys.A, sys.B, [-0.1, -0.2])
print(f"{K=}")

In [ ]:
# Design an eigenvalue placement (EP) controller to stabilize the equilibrium point
epctrl = ct.nlsys(
    None, lambda t, x, u, params: -K @ (u[0:2] - xe),
    inputs=['H', 'L', 'r'], outputs=['u'],
)
predprey_ep = ct.interconnect(
    [predprey, epctrl], inputs=['r'], outputs=['H', 'L', 'u'],
    name='predprey w/ eval placement'
)
print(predprey_ep)
predprey_ep.connection_table()

In [ ]:
xe_ep, ue_ep = ct.find_eqpt(predprey_ep, [20, 30], [0])
print(f"{xe_ep=}")
print(f"{ue_ep=}")
print("Poles: ", predprey_ep.linearize(xe_ep, ue_ep).poles())

In [ ]:
# Generate a simple phase portrait
ct.phase_plane_plot(
    predprey_ep, [0, 120, 0, 100], 1,
    # plot_streamlines=False, plot_vectorfield=True,
    # plot_equilpoints=False,
    plot_separatrices=False,
    gridtype='meshgrid', gridspec=[8, 5]
    );
ct.phaseplot.streamlines(
    predprey_ep, np.array([xe_ep]), 20, dir='reverse',
    gridtype='circlegrid', gridspec=[4, 11]);

In [ ]:
# Simulation from someplace nearby
T = np.linspace(0, 40)
response = ct.input_output_response(predprey_ep, T, 0, [35, 35])
plt.figure(figsize=[3.2, 2.4])
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response with eval placement, " +
    f"r = {predprey.params['r']}",
    legend_loc='upper right')
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')

## Integral feedback

In [ ]:
# Simulate with a change in food for the hares
T = np.linspace(0, 40)
response = ct.input_output_response(
    predprey_ep, T, 0, [35, 35], params={'r': 1.65}
)
plt.figure(figsize=[3.2, 2.4])
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response w/ eval placement, " +
    f"r = {response.params['r']}")
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')
response.sysname

In [ ]:
T = np.linspace(0, 40)
response = ct.input_output_response(
    predprey_ep, T, 0, xe, params={'r': 1.7}
)
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response for predprey w/ eval placement, " +
    f"r = {response.params['r']}")
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')
response.sysname

In [ ]:
# Integral feedback
# Design an eigenvalue placement (EP) controller to stabilize the equilibrium point
Ki = 0.0001
pictrl = ct.nlsys(
    lambda t, x, u, params: u[1] - u[2],
    lambda t, x, u, params: -K @ (u[0:2] - xe) - Ki * x[0],
    inputs=['H', 'L', 'r'], outputs=['u'], states=1,
)
predprey_pi = ct.interconnect(
    [predprey, pictrl], inputs=['r'], outputs=['H', 'L', 'u'],
    name='predprey_pi'
)
print(predprey_pi)

# Simulate with a change in food for the hares
T = np.linspace(0, 100, 500)
response = ct.input_output_response(
    predprey_pi, T, xe[1], [25, 25], params={'r': 1.65})
plt.figure(figsize=[3.2, 2.4])
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response w/ integral action, " +
    f"r = {response.params['r']}",
    legend_loc='upper right')
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')